In [1]:
import sys, os
import serial
import time
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import win32com.client
from astropy.io import fits
import threading
import nbimporter
from picoammeter_control import (
    send_cmd,
    query,
    init_cmds
)
from camera_control_ascom import (
    take_ramp,
    camera_init,
    populate_header
)
sys.path.insert(0, os.path.abspath('.'))
sys.path.insert(0, os.path.abspath('filter_wheel_control'))
from filter_wheel_control.wheel_controller import (
    list_devices,
    open_device,
    get_position,
    get_position_count,
    set_position,
    close_device
)
from data_helper_functions import (
    init_wheels,
    close_wheels,
    camera_init,
    picoam_stability,
    picoam_worker_exp,
    camera_with_picoam,
    collect_var_exp,
    var_exp_graphs,
    capture_frame,
    calculate_gain,
    get_gain,
    picoam_worker_filters,
    camera_with_filters,
    collect_var_filters,
    compare_exp_and_filter,
    collect_bias_frames
)

In [2]:
camera_init()
cam_name = "ASCOM.AtikCameras.Camera"
save_path= "C:/Users/Jonah/fits_imgs"

You selected: ASCOM.AtikCameras.Camera


In [3]:
min_exp = 0.001
max_exp = 8.0

In [13]:
init_wheels(print_stuff=False)
close_wheels()


In [6]:
e_per_adu = get_gain(cam_name=cam_name, exp_time=min_exp, num_frames=5, print_stuff=True)

starting calculation
flats collected
darks collected
Measured: 0.01957 e-/ADU on mode=176 gain
Flat mean: 989.6067975843475, Flat var: 6582.0
Dark mean: 968.8888172462242, Dark var: 5523.2


In [7]:
I_flat, Q_ramp, mean_signal, std_signal, bit_depth, exp_times, exp_low, exp_high, exp_50 = (
    collect_var_exp(cam_name=cam_name, num_exp=80, num_frames=5, exp_low = min_exp, exp_high = max_exp, save_path=save_path, offset=0)
)

var_exp_graphs(mean_signal, std_signal, bit_depth, exp_times, exp_low, exp_high, save=True, e_per_adu=e_per_adu, min_percent=0.001, percent_change=False)


Camera connected: ASCOM.AtikCameras.Camera, Size: 9576x6380, Temperature: 24.79C
Taking frames at 0.00100000 seconds (exposure time 1/80)
Taking frames at 0.00112049 seconds (exposure time 2/80)
Taking frames at 0.00125549 seconds (exposure time 3/80)
Taking frames at 0.00140676 seconds (exposure time 4/80)
Taking frames at 0.00157625 seconds (exposure time 5/80)
Taking frames at 0.00176616 seconds (exposure time 6/80)
Taking frames at 0.00197896 seconds (exposure time 7/80)
Taking frames at 0.00221740 seconds (exposure time 8/80)
Taking frames at 0.00248456 seconds (exposure time 9/80)
Taking frames at 0.00278391 seconds (exposure time 10/80)
Taking frames at 0.00311934 seconds (exposure time 11/80)
Taking frames at 0.00349517 seconds (exposure time 12/80)
Taking frames at 0.00391629 seconds (exposure time 13/80)
Taking frames at 0.00438814 seconds (exposure time 14/80)
Taking frames at 0.00491685 seconds (exposure time 15/80)
Taking frames at 0.00550926 seconds (exposure time 16/80)

KeyboardInterrupt: 

Exception in thread Thread-3:
Traceback (most recent call last):
  File "C:\Users\Jonah\AppData\Local\Programs\Python\Python39\lib\threading.py", line 973, in _bootstrap_inner
    self.run()
  File "C:\Users\Jonah\PycharmProjects\Image-Sensor-Testing-JK\venv\interpreter\lib\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "C:\Users\Jonah\AppData\Local\Programs\Python\Python39\lib\threading.py", line 910, in run
    self._target(*self._args, **self._kwargs)
  File "data_helper_functions.ipynb", line 5, in picoam_worker_exp
    "id": "initial_id",
  File "picoammeter_control.ipynb", line 9, in query
    "start_time": "2025-06-24T14:31:12.384242Z"
  File "C:\Users\Jonah\PycharmProjects\Image-Sensor-Testing-JK\venv\interpreter\lib\site-packages\serial\serialwin32.py", line 275, in read
    raise SerialException("ClearCommError failed ({!r})".format(ctypes.WinError()))
serial.serialutil.SerialException: ClearCommError failed (PermissionEr

In [14]:
signals_f, Q_filt = collect_var_filters(save_path = save_path, cam_name=cam_name, exp_50=1.7, num_frames=5, offset=0)


starting filter 1


ValueError: not enough values to unpack (expected 2, got 1)

In [ ]:
bias_level = collect_bias_frames(cam_name = cam_name,num_frames=2, exp_low=min_exp, save_path=save_path, offset=0)
print("bias level: ", bias_level)

In [ ]:
compare_exp_and_filter(e_per_adu = .025, Q_ramp=Q_ramp, Q_filt=Q_filt, signals_f=signals_f, mean_signal=mean_signal, bias_level=bias_level, save=True)
